In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

In [2]:
def split(reg):
    n = reg.size
    i = n // 2 + n % 2
    return reg[:i], reg[i:]

In [3]:
_qr = QuantumRegister(5)
B, C = split(_qr)
print(B)
print(C)

[Qubit(QuantumRegister(5, 'q0'), 0), Qubit(QuantumRegister(5, 'q0'), 1), Qubit(QuantumRegister(5, 'q0'), 2)]
[Qubit(QuantumRegister(5, 'q0'), 3), Qubit(QuantumRegister(5, 'q0'), 4)]


In [4]:
# _x = 21 # 10101

# _x % (2 ** 3)

# _x & (2 ** 3 - 1)

# _x & ((1 << 3) - 1)

# _x >> 3 

In [5]:
def get_bits(x, n):
    return [int(x) for x in '{:0{size}b}'.format(x, size=n)]

In [6]:
def generate_carry(constant, n):
    name = 'Carry ({0})'.format(constant)
    constant_bits = get_bits(constant, n)
    
    qrCtrl = QuantumRegister(1, name='ctrl')
    qrA = QuantumRegister(n, name='a') # |a> - input register
    qrC = QuantumRegister(1, name='c') # |c> - output register - information about carry
    
    # special case for n = 1: 
    if n == 1:
#         qrG = QuantumRegister(1, name='g') # for simplicity, unused
#         qc = QuantumCircuit(qrCtrl, qrA, qrG, qrC, name=name)
        qc = QuantumCircuit(qrCtrl, qrA, qrC, name=name)
    
        if constant_bits[0] == 1:
            qc.ccx(qrCtrl[0], qrA[0], qrC)
        
        return qc.to_instruction()
    
    # for n >= 2:
    qrG = QuantumRegister(n-1, name='g') # |g> - dirty ancilla register, state have to be restored
    
    qc = QuantumCircuit(qrCtrl, qrA, qrG, qrC, name=name)

    qc.ccx(qrCtrl[0], qrG[n - 2], qrC)

    for i in range(n - 1, 1, -1):
        if constant_bits[n - (i + 1)] == 1:
            qc.cx(qrA[i], qrG[i - 1])
            qc.x(qrA[i])
        qc.ccx(qrG[i - 2], qrA[i], qrG[i - 1])

    if constant_bits[n - 2] == 1:
        qc.cx(qrA[1], qrG[0])
        qc.x(qrA[1])
        
    if constant_bits[n - 1] == 1:
        qc.ccx(qrA[0], qrA[1], qrG[0])

    for i in range(2, n):
        qc.ccx(qrG[i - 2], qrA[i], qrG[i - 1])

    qc.ccx(qrCtrl[0], qrG[n - 2], qrC)
   
    for i in range(n - 1, 1, -1):
        qc.ccx(qrG[i - 2], qrA[i], qrG[i - 1])

    if constant_bits[n - 1] == 1:
        qc.ccx(qrA[0], qrA[1], qrG[0])
        
    if constant_bits[n - 2] == 1:
        qc.x(qrA[1])
        qc.cx(qrA[1], qrG[0])

    for i in range(2, n):
        qc.ccx(qrG[i - 2], qrA[i], qrG[i - 1])
        if constant_bits[n - (i + 1)] == 1:
            qc.x(qrA[i])
            qc.cx(qrA[i], qrG[i - 1])
    
    return qc.to_instruction()

In [7]:
def generate_substractor(n):
    qrCtrl = QuantumRegister(1, name='ctrl')
    qrX = QuantumRegister(n, name='g')
    qrY = QuantumRegister(n, name='a')
        
    qc = QuantumCircuit(qrCtrl, qrY, qrX)
    
    # 3.6
    for i in range(n-1, 0, -1):
        qc.cx(qrX[i], qrY[i])
        
    qc.ccx(qrCtrl[0], qrX[0], qrY[0])
    
    # 3.5
    for i in range(n-1, 1, -1):
        qc.cx(qrX[i-1], qrX[i])

    # 3.4
    for i in range(1, n):
        qc.ccx(qrX[i-1], qrY[i-1], qrX[i])
        
        qc.ccx(qrCtrl[0], qrX[i], qrY[i])

    # 3.3
    for i in range(n-1, 0, -1):
        qc.ccx(qrX[i-1], qrY[i-1], qrX[i])

    # 3.2
    for i in range(1, n-1):
        qc.cx(qrX[i], qrX[i+1])

    # 3.1
    for i in range(n-1, 0, -1):
        qc.cx(qrX[i], qrY[i])
        
    return qc

def generate_negation_part(n):
    qrG = QuantumRegister(n, name='g')
    
    qc = QuantumCircuit(qrG)
    
    qc.barrier(qrG[:])
    for i in range(n):
        qc.x(qrG[i])
    qc.barrier(qrG[:])
    
    return qc

def generate_incrementator(n):
    qc_sub = generate_substractor(n)
    qc_neg = generate_negation_part(n)
    
    qc = qc_sub + qc_neg + qc_sub + qc_neg
    qc.name = 'Inc'
    return qc.to_instruction()

In [8]:
def generate_controlled_negation(n):
    qrCtrl = QuantumRegister(1, name='ctrl')
    qrG = QuantumRegister(n, name='g')
    
    qc = QuantumCircuit(qrCtrl, qrG)
    
    qc.barrier(qrG[:])
    for i in range(n):
        qc.cx(qrCtrl[0], qrG[i])
    qc.barrier(qrG[:])
    
    return qc.to_instruction()

In [9]:
def generate_adder(constant, n):
    
    name = 'Add ({0})'.format(constant)
#     print('generating', name, n)
    
    qrCtrl = QuantumRegister(1, 'ctrl')
    qrX = QuantumRegister(n, 'x')
    qrC = QuantumRegister(1, 'carry')
    
    qc = QuantumCircuit(qrCtrl, qrX, qrC, name=name)
    
    if n == 1:
        if constant == 1:
            qc.cx(qrCtrl[0], qrX[0])
        return qc.to_instruction()
    
    i = n // 2 + n % 2
    
    low = constant & ((1 << i) - 1)
    high = constant >> i
    
#     print(i, low, high)
    
    qrX_L = qrX[:i]
    qrX_H = qrX[i:]
    
#     print(qrX_L)
#     print(qrX_H)
#     print(qrX_L[:len(qrX_H)])
    
    carry = generate_carry(low, i)
    inc = generate_incrementator(n - i)
    neg = generate_controlled_negation(n - i)
    
    qc.append(inc, qrC[:] + qrX_H[:] + qrX_L[:len(qrX_H)])
    qc.append(neg, qrC[:] + qrX_H[:])
    
    qc.append(carry, qrCtrl[:] + qrX_L[:] + qrX_H[:(i-1)] + qrC[:])
    qc.append(inc, qrC[:] + qrX_H[:] + qrX_L[:len(qrX_H)])
    qc.append(carry, qrCtrl[:] + qrX_L[:] + qrX_H[:(i-1)] + qrC[:])
    
    qc.append(neg, qrC[:] + qrX_H[:])
    
    qc.append(generate_adder(low, i), qrCtrl[:] + qrX[:i] + qrC[:])
    qc.append(generate_adder(high, n - i), qrCtrl[:] + qrX[i:] + qrC[:])
    
    return qc.to_instruction()

In [10]:
def generate_constant_substractor(constant, n):
    adder = generate_adder(constant, n)
    return adder.inverse()

In [11]:
# _qrCtrl = QuantumRegister(1, name='ctrl')
# _qrA = QuantumRegister(5, name='a')
# _qrC = QuantumRegister(1, name='carry')
# _qc = QuantumCircuit(_qrCtrl, _qrA, _qrC)

# adder = generate_constant_substractor(21, 5)
# _qc.append(adder, _qrCtrl[:] + _qrA[:] + _qrC[:])
# _qc.draw(output='mpl')

In [12]:
# _dec = _qc.decompose()
# _dec.draw(output='mpl')

In [13]:
def generate_init_part(qc, ctrl, a, g, n):
    [qrCtrl, qrA, qrC] = qc.qregs
    
    a_bits = get_bits(a, n)
        
    if ctrl:
        qc.x(qrCtrl[0])
    
    for i in range(n):
        if a_bits[n-(i+1)] == 1:
            qc.x(qrA[i])

    if g == 1:
        qc.x(qrC[0])
            
    qc.barrier(qrCtrl[:], qrA[:])

In [14]:
# _qrCtrl = QuantumRegister(1, name='ctrl')
# _qrA = QuantumRegister(3, name='a')
# _qrC = QuantumRegister(1, name='carry')
# _qc = QuantumCircuit(_qrCtrl, _qrA, _qrC)

# generate_init_part(_qc, True, 5, 3)
# _qc.draw(output='mpl')

In [15]:
def generate_measure_part(n):
    qrX = QuantumRegister(n, name='x')
    qrC = QuantumRegister(1, name='carry')
    crX = ClassicalRegister(n, name='xValue')
    crC = ClassicalRegister(1, name='cValue')

    qc = QuantumCircuit(qrX, qrC, crX, crC)
    qc.barrier(qrX[:], qrC[:])
    qc.measure(qrX[:], crX[:])
    qc.measure(qrC[:], crC[:])
    
    return qc

In [16]:
# qc_meas = generate_measure_part(3)
# qc_meas.draw(output='mpl')

In [17]:
from qiskit import Aer
from qiskit import execute

backend = Aer.get_backend('qasm_simulator')

In [18]:
def test_substraction(ctrl, a, constant, n, a_value):
    expected_value = (a - constant) % (2 ** n) if ctrl else a
    print(expected_value, a_value, expected_value == a_value)

In [19]:
test_substraction(True, 14, 21, 5, 25)
test_substraction(False, 14, 21, 5, 14)

25 25 True
14 14 True


In [20]:
def test(ctrl, x, constant, g, n):
    qrCtrl = QuantumRegister(1, name='ctrl')
    qrX = QuantumRegister(n, name='x')
    qrC = QuantumRegister(1, 'carry')
    
    qc = QuantumCircuit(qrCtrl, qrX, qrC)
    
    generate_init_part(qc, ctrl, x, g, n)
    qc.append(generate_constant_substractor(constant, n), qrCtrl[:] + qrX[:] + qrC[:])
    qc += generate_measure_part(n)

    job = execute(qc, backend, shots=1)
    result = job.result()
    values = list(result.get_counts(qc).keys())

    assert len(values) == 1
    values = [int(v, 2) for v in values[0].split(' ')]

    c_value = values[0]
    x_value = values[1]

    test_substraction(ctrl, x, constant, n, x_value)
    print(g, c_value, g == c_value)

In [21]:
test(True, 14, 5, 0, 4)
test(True, 14, 5, 1, 4)

<ipython-input-7-7e1d57df6cd4>:54: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  qc = qc_sub + qc_neg + qc_sub + qc_neg
/home/bartek/S10/MGR/masterThesisProject/venv/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py:869: DeprecationWarning: The QuantumCircuit.combine() method is being deprecated. Use the compose() method which is more flexible w.r.t circuit register compatibility.
  return self.combine(rhs)
<ipython-input-20-9441eefa58f0>:10: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  qc += generate_measure_part(n)
/home/bartek/S10/MGR/masterThesisProject/venv/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py:876: DeprecationWarning: The QuantumCircuit.extend() method 

9 9 True
0 0 True
9 9 True
1 1 True


In [22]:
# for n in range(5):
#     n = n + 1
#     print(n)
#     N = 2 ** n
#     for ctrl in range(2):
#         for a in range(N):
#             for c in range(N):
#                 for g in range(2):
#                     print('---', ctrl, '-', a, c, g, '---')
#                     test(ctrl == 1, a, c, g, n)